In [1]:
from app.tools.db import DatabaseConnection
from app.tools.constans import paramsl, paramsp
from app.tools.s3 import download_file
import ijson, json, uuid

connl = DatabaseConnection(**paramsl)
connl.connect()

connp = DatabaseConnection(**paramsp)
connp.connect()

def generate_uid():
    uid = uuid.uuid4()
    return str(uid)

In [4]:
s3_path = "ajio-myntra/alternative/20240201/dev/brands_dictionary_women.json"
local_path = 
file_name = ""

download_file(s3_path, local_path, file_name)

El archivo brands_dictionary_women.json fue descargado en la ruta:
trash/dicts


In [5]:
query = "SELECT id, origin_id, alternative_id, similarity FROM request_e07aaf8f_a587_42af_a0b3_0abd42a7ffc5.matchings"
query2 = "SELECT products FROM request_e07aaf8f_a587_42af_a0b3_0abd42a7ffc5.{table} WHERE id = {id}"
query3 = 'INSERT INTO public."ProductsRequest"(id, "idRequest", "originProducts", distance, "alternativeProducts", resource) VALUES(%s,%s,%s,%s,%s,%s)'

connl.execute(query)
result_matchings = connl.result.fetchall()

for data in result_matchings:

    origin_id = data[1]
    alternative_id = data[2]
    similarity = data[3]
    idRequest = "6ac49d9c-0d9d-4c81-9591-4aac9219f329"

    connl.execute(query2.format(table="origin", id=origin_id))
    product_origin = json.dumps(connl.result.fetchone()[0])

    connl.execute(query2.format(table="alternative", id=alternative_id))
    product_alternative = json.dumps(connl.result.fetchone()[0])

    id_ = generate_uid()

    resource = json.dumps({
        "path_origin": "ajio-myntra/origin/20240131/",
        "path_alternative": ["ajio-myntra/alternative/20240131/dev/", "ajio-myntra/alternative/20240202/dev/", "ajio-myntra/alternative/20240131/dev/"]
    })

    params = (id_, idRequest, product_origin, similarity, product_alternative, resource)

    connp.execute(query3, params)

connp.commit()

